In [1]:
#%%
import numpy as np
import healpy as hp
import math
import time
import normsandangles
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from pylab import arange, show, cm
import scipy
from scipy import special

%matplotlib inline
mpl.rcParams.update({'font.size':12})

#https://stackoverflow.com/questions/34023932/mollview-use-matplotlib-colormaps-and-change-background-color
cmap = cm.jet #stores old healpy colloring scheme (cmap='jet')
cmap.set_under('w')

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [2]:
def phi_of_k_cart(norms_and_vecs):
    """
    Dictionary storing norms as keys and random \phi(\vec{k}) as values. We have 8 independent 
    entries for \phi, each for one quadrant of the Fourier sphere, and each entry having 
    length equal to the number of vectors \hat{k} for a fixed norm k.
    """
    phi_rand = []
    phi_k_dict = {}
    for norm in norms_and_vecs.keys():
        phi_rand = np.random.randn(8,len(norms_and_vecs[norm]))#we need 8 independent random vectors. See pdf for details
        phi_k_dict.update({norm:phi_rand})
    return phi_k_dict

In [3]:
#%%
def phi_of_x_cart(Lsize,x,y,z,norms_and_vecs,Phi_k_dict,alpha):
    """
    Scalar function returning the gravitational potential for a given pixel with coodinates (x,y,z). 
    For details see the pdf. Currently it only works for toroidal topology.
    alpha = ratio between Lx/Lz and Ly/Lz (quasi-slab topology)
    """
    
    Phi_x = 0.0 #Initial value of \Phi(\vec{x})
    prefactor = 0.0
    R = 1.0 #Radius of the CMB surface
    A = 1.0 #Amplitude of the Power Spectrum
    
    for norm in norms_and_vecs.keys():
        
        k = 2.0*math.pi*norm/Lsize/alpha#norm of Fourier vector
        sqrt_Pspec = math.sqrt(A*(k**(-3.0)))#Square-root of Harrison-Zeldovich Power-spectrum
        khat = np.transpose(norms_and_vecs[norm])# unit Fourier vectors
        
        cos_gamma1 = np.dot(np.array([x,y,z]),khat) 
        # dot of \hat{x} and \hat{k} in exp(ix.k) in the first octant        
        cos_gamma2 = np.dot(np.array([x,-y,z]),khat) 
        # dot of \hat{x} and \hat{k} in exp(ix.k) in the 2nd octant: y -> - y        
        cos_gamma3 = np.dot(np.array([-x,-y,z]),khat) 
        # dot of \hat{x} and \hat{k} in exp(ix.k) in the 3rd octant: x, y -> -x, -y        
        cos_gamma4 = np.dot(np.array([-x,y,z]),khat) 
        # dot of \hat{x} and \hat{k} in exp(ix.k) in the 4th octant: x -> -x 
        
        prefactor = 2*(np.cos(k*R*cos_gamma1)*Phi_k_dict[norm][0]-np.sin(k*R*cos_gamma1)*Phi_k_dict[norm][1] + \
                    np.cos(k*R*cos_gamma2)*Phi_k_dict[norm][2]-np.sin(k*R*cos_gamma2)*Phi_k_dict[norm][3] + \
                    np.cos(k*R*cos_gamma3)*Phi_k_dict[norm][4]-np.sin(k*R*cos_gamma3)*Phi_k_dict[norm][5] + \
                    np.cos(k*R*cos_gamma4)*Phi_k_dict[norm][6]-np.sin(k*R*cos_gamma4)*Phi_k_dict[norm][7])
        prefactor = np.sum(prefactor) #sum over all angles with fixed nnorm
        
        Phi_x += sqrt_Pspec*prefactor #sum over norms
    return Phi_x

In [4]:
#%%
def phi_map_cart(Nside,nmax,Lsize,alpha,seed):
    np.random.seed(seed) # fixes random seed, for debugging
    normvecs_dict = normsandangles.vector_n(nmax,alpha) #initialize norms and vectors
    randphi_dict = phi_of_k_cart(normvecs_dict) #initialize random values of \phi(\vec{k})
    phi_in_pixel = []
    pixels = np.arange(hp.nside2npix(Nside))
    for x,y,z in np.transpose(hp.pix2vec(Nside,pixels)):
        phi_in_pixel.append(phi_of_x_cart(Lsize,x,y,z,normvecs_dict,randphi_dict,alpha))
    return np.array(phi_in_pixel)

In [5]:
%time slab0p1ns64_cart = phi_map_cart(4,6,0.1,4,1)

CPU times: user 901 ms, sys: 29 µs, total: 901 ms
Wall time: 901 ms


In [6]:
hp.mollview(slab0p1ns64_cart,rot=[0.,50.,60.])

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error ../3ebdb3b56fb83dd7122652c4cb96308c.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(../3ebdb3b56fb83dd7122652c4cb96308c.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.7 \usepackage
               {type1ec}^^M
No pages of output.
Transcript written on 3ebdb3b56fb83dd7122652c4cb96308c.log.




RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error ../3ebdb3b56fb83dd7122652c4cb96308c.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(../3ebdb3b56fb83dd7122652c4cb96308c.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.7 \usepackage
               {type1ec}^^M
No pages of output.
Transcript written on 3ebdb3b56fb83dd7122652c4cb96308c.log.




<Figure size 612x388.8 with 2 Axes>